# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [3]:
import os
import sys

sys.path.append('..')
# Set your preferred name
USER = "Yuxuan"
# Set the data and output directories
DATA_ROOT = '/home/yxcheng'
OUTPUT_ROOT = '../../output1'
TRAIT = 'Hypertension'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [4]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:]

['TCGA_Adrenocortical_Cancer_(ACC)',
 'TCGA_Bile_Duct_Cancer_(CHOL)',
 'TCGA_Lower_Grade_Glioma_(LGG)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Esophageal_Cancer_(ESCA)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Ocular_melanomas_(UVM)',
 'TCGA_Ovarian_Cancer_(OV)',
 'TCGA_Prostate_Cancer_(PRAD)',
 'CrawlData.ipynb',
 'TCGA_Kidney_Papillary_Cell_Carcinoma_(KIRP)',
 'TCGA_Lung_Cancer_(LUNG)',
 'TCGA_Endometrioid_Cancer_(UCEC)',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)',
 'TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Glioblastoma_(GBM)',
 'TCGA_Melanoma_(SKCM)',
 'TCGA_Kidney_Clear_Cell_Carcinoma_(KIRC)',
 'TCGA_Mesothelioma_(MESO)',
 'TCGA_Rectal_Cancer_(READ)',
 'TCGA_Head_and_Neck_Cancer_(HNSC)',
 'TCGA_Large_Bcell_Lymphoma_(DLBC)',
 'TCGA_Uterine_Carcinosarcoma_(UCS)',
 'TCGA_Colon_Cancer_(COAD)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Bladder_Cancer_(BLCA)',
 'TCGA_Sarcoma_(SARC)',
 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)',
 'TCGA_Cervical_Ca

If no match is found, jump directly to GEO in Part 2.2

In [27]:
trait_subdir = "TCGA_Lung_Cancer_(LUNG)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_LUNG_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_HiSeqV2_PANCAN.gz')

In [28]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [ ]:
clinical_data_cols[:10]

['ABSOLUTE_Ploidy',
 'ABSOLUTE_Purity',
 'AKT1',
 'ALK_translocation',
 'BRAF',
 'CBL',
 'CTNNB1',
 'Canonical_mut_in_KRAS_EGFR_ALK',
 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2',
 'EGFR']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['ABSOLUTE_Ploidy', 'ABSOLUTE_Purity', 'AKT1', 'ALK_translocation', 'BRAF', 'CBL', 'CTNNB1', 'Canonical_mut_in_KRAS_EGFR_ALK', 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2', 'EGFR', 'ERBB2', 'ERBB4', 'Estimated_allele_fraction_of_a_clonal_varnt_prsnt_t_1_cpy_pr_cll', 'Expression_Subtype', 'HRAS', 'KRAS', 'MAP2K1', 'MET', 'NRAS', 'PIK3CA', 'PTPN11', 'Pathology', 'Pathology_Updated', 'RET_translocation', 'ROS1_translocation', 'STK11', 'WGS_as_of_20120731_

In [ ]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'stopped_smoking_year', 'year_of_initial_pathologic_diagnosis', 'year_of_tobacco_smoking_onset']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [70.0, 67.0, 79.0, 68.0, 66.0],
 'days_to_birth': [-25752.0, -24532.0, -29068.0, -24868.0, -24411.0],
 'days_to_collection': [nan, nan, nan, nan, nan],
 'days_to_death': [nan, nan, '121', nan, nan],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, 0.0],
 'days_to_last_followup': ['0', '1523', nan, '607', '426'],
 'days_to_new_tumor_event_after_initial_treatment': [nan,
  nan,
  nan,
  334.0,
  183.0],
 'stopped_smoking_year': [nan, nan, nan, nan, 1987.0],
 'year_of_initial_pathologic_diagnosis': [2009.0,
  2007.0,
  2007.0,
  2009.0,
  2009.0],
 'year_of_tobacco_smoking_onset': [nan, nan, nan, nan, 1963.0]}

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Lung-Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-05-4244-01,1,70.0,1.0,-0.056592,2.128674,-0.531035,-0.185972,-0.190878,-0.11891,-0.659794,...,0.015022,-1.155117,0.528367,2.75061,0.428423,-0.639586,-0.195527,-0.324695,0.193167,-0.572812
TCGA-05-4249-01,1,67.0,1.0,-0.569692,-0.318126,-0.531035,0.237428,0.011822,-0.00611,1.018406,...,-0.031778,-0.393117,1.632467,2.33551,-0.145577,-0.997486,-0.276027,-1.219395,1.850967,-0.471212
TCGA-05-4250-01,1,79.0,0.0,-0.899992,-0.918126,-0.531035,-0.621472,-0.454178,0.58719,-0.198894,...,-0.748878,-1.173917,1.966367,-1.28139,-0.374877,0.959514,0.714373,0.112305,-0.532833,-0.003812
TCGA-05-4382-01,1,68.0,1.0,0.667508,3.895974,-0.221535,-0.367272,0.131822,0.97629,-0.494994,...,1.013722,-1.587117,0.359267,2.59051,0.806623,0.812114,0.367373,-0.494595,-1.062233,0.108588
TCGA-05-4384-01,1,66.0,1.0,-0.316892,2.380374,0.156865,0.232828,0.054822,0.27059,0.128606,...,-0.060978,-0.435417,1.111467,-1.28139,-0.128477,0.110214,1.469373,0.194605,1.404667,-0.290712


In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 1100 samples.


In [ ]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Lung-Cancer', the least common label is '0' with 110 occurrences. This represents 10.00% of the dataset.
The distribution of the feature 'Lung-Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 60.0
  50% (Median): 67.0
  75%: 73.0
Min: 38.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 443 occurrences. This represents 40.27% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Yuxuan/Lung-Cancer/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [5]:
dataset = 'GEO'
trait_subdir = "Hypertension"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE181339', 'GSE205138', 'GSE207015']

Repeat the below steps for all the accession numbers

In [14]:
cohort = accession_num = "GSE207015"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/home/yxcheng/GEO/Hypertension/GSE207015/GSE207015_family.soft.gz',
 '/home/yxcheng/GEO/Hypertension/GSE207015/GSE207015_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [15]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"COVID-19 drives a distinct plasma proteome in pregnant and non-pregnant individuals"
!Series_summary	"This study used SomaScan v 4.1 to profile>7000 proteins in human plasma and assess changes with Sars-Cov-2 infection and with Covid-19 disease severity in pregnant and non-pregnant individuals."
!Series_overall_design	"This is a cross-sectional study that included non infected controls and Sars-Cov-2 infected patients with various degrees of disease serverity."


In [20]:
clinical_data.head()

,!Sample_geo_accession,GSM6268053,GSM6268054,GSM6268055,GSM6268056,GSM6268057,GSM6268058,GSM6268059,GSM6268060,GSM6268061,...,GSM6268237,GSM6268238,GSM6268239,GSM6268240,GSM6268241,GSM6268242,GSM6268243,GSM6268244,GSM6268245,GSM6268246
0,!Sample_characteristics_ch1,individual: PNI087001,individual: CPN019001,individual: PNI015001,individual: PNI049001,individual: PNI072001,individual: PNI060001,individual: CPN031001,individual: PNI114001,individual: PNI093001,...,individual: CWNIH010,individual: CWNIH012,individual: CWNIH013,individual: CWNIH014,individual: CWNIH015,individual: CWNIH016,individual: CWNIH017,individual: CWNIH018,individual: CWNIH019,individual: CWNIH020
1,!Sample_characteristics_ch1,age: 23,age: 34,age: 32,age: 36,age: 29,age: 18,age: 18,age: 29,age: 34,...,age: 58,age: 58,age: 63,age: 32,age: 25,age: 34,age: 68,age: 62,age: 68,age: 25
2,!Sample_characteristics_ch1,bmi: 23.2,bmi: 27.6,bmi: 32,bmi: 24.5,bmi: 34.5,bmi: 26.1,bmi: 25.5,bmi: 32.5,bmi: 28,...,bmi: 22,bmi: 28.2,bmi: 33.1,bmi: 22,bmi: 28.7,bmi: 26.1,bmi: 26.6,bmi: 26,bmi: 31.6,bmi: 26.8
3,!Sample_characteristics_ch1,chronic hypertension: No,chronic hypertension: Yes,chronic hypertension: No,chronic hypertension: No,chronic hypertension: No,chronic hypertension: No,chronic hypertension: No,chronic hypertension: No,chronic hypertension: No,...,chronic hypertension: No,chronic hypertension: Yes,chronic hypertension: Yes,chronic hypertension: No,chronic hypertension: No,chronic hypertension: No,chronic hypertension: Yes,chronic hypertension: No,chronic hypertension: No,chronic hypertension: Yes
4,!Sample_characteristics_ch1,Sex: Female,Sex: Female,Sex: Female,Sex: Female,Sex: Female,Sex: Female,Sex: Female,Sex: Female,Sex: Female,...,Sex: Female,Sex: Female,Sex: Female,Sex: Female,Sex: Female,Sex: Female,Sex: Female,Sex: Female,Sex: Female,Sex: Female


In [19]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['individual: PNI087001',
  'individual: CPN019001',
  'individual: PNI015001',
  'individual: PNI049001',
  'individual: PNI072001',
  'individual: PNI060001',
  'individual: CPN031001',
  'individual: PNI114001',
  'individual: PNI093001',
  'individual: PNI089001',
  'individual: PNI062001',
  'individual: PNI032001',
  'individual: PNI034002',
  'individual: PNI111001',
  'individual: CPN010001',
  'individual: PNI110001',
  'individual: PNI002001',
  'individual: PNI088001',
  'individual: PNI068001',
  'individual: PNI014001',
  'individual: PNI043001',
  'individual: PNI046001',
  'individual: PNI120001',
  'individual: PNI036001',
  'individual: PNI097001',
  'individual: PNI055001',
  'individual: PNI119001',
  'individual: PNI053001',
  'individual: PNI031001',
  'individual: PNI045001'],
 1: ['age: 23',
  'age: 34',
  'age: 32',
  'age: 36',
  'age: 29',
  'age: 18',
  'age: 28',
  'age: 33',
  'age: 41',
  'age: 37',
  'age: 27',
  'age: 21',
  'age: 35',
  'age: 39',
 

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [33]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Psoriasis\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Psoriasis\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an intege

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [21]:
is_gene_availabe = True
trait_row = 3
age_row = 1
gender_row = 4

trait_type = 'binary'

In [22]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [23]:
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if tissue_type == 'chronic hypertension: Yes':
        return 1  # Epilepsy present
    else:
        return 0  # Epilepsy not present

def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age: '):
        try:
            return int(age_string.split('age: ')[1].rstrip(''))  # Extracting numeric part
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format

def convert_gender(gender_string):
    """
    Convert gender string to a binary representation.
    Female -> 0, Male -> 1, Unknown -> None
    """
    if gender_string == 'Sex: Female':
        return 0
    elif gender_string == 'Sex: Male':
        return 1
    else:
        return None  # In case of unknown gender

# Example usage
example_tissue = 'tissue: Hippocampus'
example_age = 'age: 23y'
example_gender = 'gender: Male'

converted_trait = convert_trait(example_tissue)
converted_age = convert_age(example_age)
converted_gender = convert_gender(example_gender)

converted_trait, converted_age, converted_gender

(0, None, None)

In [24]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

/home/yxcheng/AI4Science_gold_standard/stage1/YIjiang/../utils.py:208: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)
/home/yxcheng/AI4Science_gold_standard/stage1/YIjiang/../utils.py:208: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)
/home/yxcheng/AI4Science_gold_standard/stage1/YIjiang/../utils.py:208: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)


,GSM6268053,GSM6268054,GSM6268055,GSM6268056,GSM6268057,GSM6268058,GSM6268059,GSM6268060,GSM6268061,GSM6268062,...,GSM6268237,GSM6268238,GSM6268239,GSM6268240,GSM6268241,GSM6268242,GSM6268243,GSM6268244,GSM6268245,GSM6268246
Hypertension,0,1,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,1,0,0,1
Age,23,34,32,36,29,18,18,29,34,28,...,58,58,63,32,25,34,68,62,68,25
Gender,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Genetic data preprocessing and final filtering

In [25]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM6268053,GSM6268054,GSM6268055,GSM6268056,GSM6268057,GSM6268058,GSM6268059,GSM6268060,GSM6268061,GSM6268062,...,GSM6268237,GSM6268238,GSM6268239,GSM6268240,GSM6268241,GSM6268242,GSM6268243,GSM6268244,GSM6268245,GSM6268246
ID,,,,,,,,,,,,,,,,,,,,,
A1BG.16561.9,11.202246,11.207136,10.426998,11.173365,11.010388,10.933986,11.123345,10.727325,10.966866,11.104599,...,10.907191,11.041522,10.683609,10.546219,10.323167,10.745758,11.066426,10.849170,10.322041,10.770168
A1CF.12423.38,10.481699,10.417114,10.795877,10.951430,10.082016,10.290595,10.873290,10.687201,10.794091,12.203379,...,10.225448,9.634811,9.708394,9.804776,10.227616,9.840148,9.760720,9.771820,9.695402,13.576189
A2M.3708.62,14.462036,13.766643,14.160848,14.046126,13.529931,13.476696,14.154715,13.102615,13.900339,14.349254,...,13.635310,13.305449,13.750749,13.891926,14.377305,13.716755,13.310499,13.777779,12.418248,13.683444
A2ML1.15482.12,11.501240,9.385431,9.401733,9.282857,8.822730,9.000282,11.093021,8.976134,9.367633,9.392747,...,14.534370,9.979568,9.094342,9.599727,9.482808,9.161888,9.384784,9.029011,9.120756,9.109047
A4GALT.8759.29,9.506605,8.936638,8.972693,8.960292,8.149747,8.414474,9.221104,8.379378,8.976134,9.237210,...,8.455327,10.254320,8.974988,9.244126,9.071194,8.983849,9.185123,8.812498,8.381110,9.057992


In [26]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['A1BG.16561.9',
 'A1CF.12423.38',
 'A2M.3708.62',
 'A2ML1.15482.12',
 'A4GALT.8759.29',
 'A4GNT.10835.25',
 'AACS.21515.61',
 'AADAT.22023.3',
 'AAGAB.11926.23',
 'AAMDC.24216.30',
 'AARS1.12340.17',
 'AARSD1.21535.5',
 'AASDHPPT.17320.19',
 'ABCC6.8935.22',
 'ABCD4.25886.11',
 'ABCF3.23700.42',
 'ABHD10.17370.186',
 'ABHD12.7825.7',
 'ABHD14A.5715.4',
 'ABHD14B.19325.21']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [27]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['A1BG.16561.9', 'A1CF.12423.38', 'A2M.3708.62', 'A2ML1.15482.12', 'A4GALT.8759.29', 'A4GNT.10835.25', 'AACS.21515.61', 'AADAT.22023.3', 'AAGAB.11926.23', 'AAMDC.24216.30', 'AARS1.12340.17', 'AARSD1.21535.5', 'AASDHPPT.17320.19', 'ABCC6.8935.22', 'ABCD4.25886.11', 'ABCF3.23700.42', 'ABHD10.17370.186', 'ABHD12.7825.7', 'ABHD14A.5715.4', 'ABHD14B.19325.21']\n"

If not required, jump directly to the gene normalization step

In [28]:
requires_gene_mapping = True

In [29]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['CRYBB2.10000.28', 'RAF1.10001.7', 'ZNF41.10003.15', 'ELK1.10006.25', 'GUCA1A.10008.43'], 'SeqId': ['10000-28', '10001-7', '10003-15', '10006-25', '10008-43'], 'SeqIdVersion': [3.0, 3.0, 3.0, 3.0, 3.0], 'SomaId': ['SL019233', 'SL002564', 'SL019245', 'SL019228', 'SL019234'], 'TargetFullName': ['Beta-crystallin B2', 'RAF proto-oncogene serine/threonine-protein kinase', 'Zinc finger protein 41', 'ETS domain-containing protein Elk-1', 'Guanylyl cyclase-activating protein 1'], 'Target': ['CRBB2', 'c-Raf', 'ZNF41', 'ELK1', 'GUC1A'], 'UniProt': ['P43320', 'P04049', 'P51814', 'P19419', 'P43080'], 'EntrezGeneID': ['1415', '5894', '7592', '2002', '2978'], 'EntrezGeneSymbol': ['CRYBB2', 'RAF1', 'ZNF41', 'ELK1', 'GUCA1A'], 'Organism': ['Human', 'Human', 'Human', 'Human', 'Human'], 'Units': ['RFU', 'RFU', 'RFU', 'RFU', 'RFU'], 'Type': ['Protein', 'Protein', 'Protein', 'Protein', 'Protein'], 'SPOT_ID': ['SL019233', 'SL002564', 'SL019245', 'SL019228', 'SL019234']}


Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [30]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['CRYBB2.10000.28', 'RAF1.10001.7', 'ZNF41.10003.15', 'ELK1.10006.25', 'GUCA1A.10008.43'], 'SeqId': ['10000-28', '10001-7', '10003-15', '10006-25', '10008-43'], 'SeqIdVersion': [3.0, 3.0, 3.0, 3.0, 3.0], 'SomaId': ['SL019233', 'SL002564', 'SL019245', 'SL019228', 'SL019234'], 'TargetFullName': ['Beta-crystallin B2', 'RAF proto-oncogene serine/threonine-protein kinase', 'Zinc finger protein 41', 'ETS domain-containing protein Elk-1', 'Guanylyl cyclase-activating protein 1'], 'Target': ['CRBB2', 'c-Raf', 'ZNF41', 'ELK1', 'GUC1A'], 'UniProt': ['P43320', 'P04049', 'P51814', 'P19

In [31]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'EntrezGeneSymbol'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [32]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

1 input query terms found dup hits:	[('BAGE2', 2)]
10 input query terms found no hit:	['ANP32C', 'BAP18', 'C11orf49', 'C12orf65', 'C16orf71', 'C16orf72', 'C19orf54', 'C5orf38', 'C7orf69'
1 input query terms found dup hits:	[('DNAJB3', 2)]
9 input query terms found no hit:	['DDX58', 'DUPD1', 'DUSP13', 'EFCAB1', 'EURL', 'FAM102B', 'FAM160B1', 'FAM172A', 'FAM189A2']
10 input query terms found dup hits:	[('HBD', 2), ('IGHA1', 2), ('IGHA2', 2), ('IGHD', 2), ('IGHE', 2), ('IGHG1', 2), ('IGHG2', 2), ('IGH
7 input query terms found no hit:	['FAM92A', 'FLJ44635', 'GBA', 'GGT2', 'H2AW', 'H2BU1', 'HHLA3']
9 input query terms found no hit:	['LAS2', 'LEG1', 'MAGEA5', 'MENT', 'MPP5', 'MPP6', 'MYLPF', 'NMES1', 'NRAC']
3 input query terms found dup hits:	[('SIGLEC5', 2), ('TCP10L3', 2), ('TEC', 2)]
6 input query terms found no hit:	['SARG', 'SLC9A3R1', 'SPATA5', 'TDGF1', 'TSTA3', 'TTC25']
1 input query terms found dup hits:	[('TYMSOS', 2)]
2 input query terms found no hit:	['TXNRD3NB', 'ZADH2']


In [33]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [34]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 194 samples.


In [35]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Hypertension', the least common label is '1.0' with 53 occurrences. This represents 27.32% of the dataset.
The distribution of the feature 'Hypertension' in this dataset is fine.

Quartiles for 'Age':
  25%: 28.0
  50% (Median): 35.5
  75%: 57.75
Min: 16.0
Max: 85.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 44 occurrences. This represents 22.68% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [36]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

A new JSON file was created at: ../../output1/Yuxuan/Hypertension/cohort_info.json


In [37]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [38]:
from utils import *

In [39]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

'GSE207015'

In [40]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE207015,True,True,False,True,True,194,


In [41]:
ck=100
listq=[]
for df in pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'),chunksize=ck):
    listq.append(df)
merged_data = pd.concat(listq)
#merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [42]:
merged_data.head()

,Hypertension,Age,Gender,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AACS,...,ZP4,ZPBP,ZPBP2,ZPLD1,ZRANB1,ZSWIM7,ZW10,ZWILCH,ZWINT,ZYX
0,0.0,23.0,0.0,11.202246,10.481699,14.462036,11.501240,9.506605,9.536247,10.440869,...,10.449665,11.465362,5.889960,9.269828,10.495655,8.822093,12.875404,11.514320,10.832099,14.440649
1,1.0,34.0,0.0,11.207136,10.417114,13.766643,9.385431,8.936638,10.080018,9.851749,...,10.443254,11.665691,5.951868,9.280075,10.116864,8.731998,12.718704,11.920874,10.926741,14.629698
2,0.0,32.0,0.0,10.426998,10.795877,14.160848,9.401733,8.972693,9.459227,9.207502,...,10.239360,11.642954,6.123087,9.475531,9.081350,8.866661,13.817613,11.538771,10.524248,11.585808
3,0.0,36.0,0.0,11.173365,10.951430,14.046126,9.282857,8.960292,10.177420,9.693313,...,10.394570,11.491001,6.011227,9.435670,9.810572,8.781360,12.481875,11.457022,10.685800,13.942469
4,0.0,29.0,0.0,11.010388,10.082016,13.529931,8.822730,8.149747,9.485226,9.278449,...,10.235775,11.550699,5.956521,9.401306,9.400666,8.659282,12.754136,11.274203,10.301153,13.308239


In [43]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [44]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [45]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [46]:

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [47]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 43.16% ± 3.57%


In [48]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

The cross-validation accuracy is 46.84% ± 4.53%


In [52]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


### 5. Discussion and report

In [50]:

feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.2864
Found 130 genes with non-zero coefficients associated with the trait 'Hypertension' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
Variable  Coefficient  Absolute Coefficient
 ST8SIA4    -2.745913              2.745913
    DPP4     2.509702              2.509702
   CRLF2     2.413312              2.413312
   NAGLU     1.903141              1.903141
  KLHL14     1.794163              1.794163
  EPS8L3    -1.783291              1.783291
     GK5     1.730039              1.730039
  FRRS1L     1.670916              1.670916
    BTLA    -1.648225              1.648225
  SEMA6A     1.644073              1.644073
   RALYL     1.613170              1.613170
    USF1    -1.417886              1.417886
  GLT8D1     1.399191              1.399191
   PRPF6     1.302602              1.302602
 DNAJC15    -1.280723              1.280723
    SGSH     1.174956   

/home/yxcheng/AI4Science_gold_standard/stage1/YIjiang/../utils.py:469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [51]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.2864
Found 0 significant genes associated with the trait 'Hypertension' conditional on the factor 'Age', with corrected p-value < 0.05:
Empty DataFrame
Columns: [Variable, Coefficient, p_value, corrected_p_value]
Index: []


/home/yxcheng/AI4Science_gold_standard/stage1/YIjiang/../utils.py:478: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
